# Finding the Best Hotel for the Perfect Charleston Vacation
### by Angel Fitzgerald, Capstone Project for the IBM Applied Data Science Certification

## Getting ZIP Codes and Associated Latitudes & Longitudes from Websites with BeautifulSoup
### ZIP Code Acquisition and Data Cleaning
To begin our exploration of the Charleston area, we will first use BeautifulSoup to scrape ZIP Codes from https://www.zip-codes.com/county/sc-charleston.asp.  This information will be transformed into a pandas dataframe.

In [1]:
#Import necessary libraries and webpage

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
import requests

page = requests.get('https://www.zip-codes.com/county/sc-charleston.asp#zipcodes').text
soup = BeautifulSoup(page, "lxml")

In [2]:
#Scrape table from web page

table = soup.find('table', class_='statTable')

In [3]:
#Get ZIP Code Data from table and store in list z

z = []

for zipcode in table.find_all('td', class_='label'):
    z.append(zipcode.text)

print(z)

['ZIP Code', 'ZIP Code 29401', 'ZIP Code 29402', 'ZIP Code 29403', 'ZIP Code 29404', 'ZIP Code 29405', 'ZIP Code 29406', 'ZIP Code 29407', 'ZIP Code 29409', 'ZIP Code 29412', 'ZIP Code 29413', 'ZIP Code 29414', 'ZIP Code 29415', 'ZIP Code 29416', 'ZIP Code 29417', 'ZIP Code 29418', 'ZIP Code 29419', 'ZIP Code 29422', 'ZIP Code 29423', 'ZIP Code 29424', 'ZIP Code 29425', 'ZIP Code 29426', 'ZIP Code 29429', 'ZIP Code 29439', 'ZIP Code 29449', 'ZIP Code 29451', 'ZIP Code 29455', 'ZIP Code 29457', 'ZIP Code 29458', 'ZIP Code 29464', 'ZIP Code 29465', 'ZIP Code 29466', 'ZIP Code 29470', 'ZIP Code 29482', 'ZIP Code 29487']


Now that the ZIP Code data is in a list, we can begin building and cleaning the pandas dataframe.

In [4]:
#Create a pandas dataframe from the ZIP Code list

df = pd.DataFrame(z) 
df.head()

,0
0,ZIP Code
1,ZIP Code 29401
2,ZIP Code 29402
3,ZIP Code 29403
4,ZIP Code 29404


In [5]:
#The first row should be the label for the dataframe column.

new_header = df.iloc[0] #grab the first row for the label
df = df[1:] #take the data less the header row
df.columns = new_header #set the label row as the df label
df.head()

,ZIP Code
1,ZIP Code 29401
2,ZIP Code 29402
3,ZIP Code 29403
4,ZIP Code 29404
5,ZIP Code 29405


In [6]:
#Finally, we should remove the text "ZIP Code" from each of the dataframe elements.  We do this by removing the first 8 characters of each ZIP Code element.

df['ZIP Code'] = df['ZIP Code'].map(lambda x: str(x)[8:])
df.head()

,ZIP Code
1,29401
2,29402
3,29403
4,29404
5,29405


In [7]:
#According to the ZIP Codes website, we should have 34 ZIP Codes for Charleston.  Let's verify that by checking the shape of the dataframe.

df.shape

(34, 1)

It looks like we have the correct number of ZIP Codes for Charleston.

In [8]:
#Now, let's look at the data type for ZIP Code.

df.dtypes

0
ZIP Code    object
dtype: object

In [9]:
#Convert ZIP Code to int64.  Our ZIP Code data type will need to match for both the dataframe and the latitude & longitude data.

df = pd.to_numeric(df['ZIP Code'])
df.dtypes

dtype('int64')

### Latitude & Longitude .csv Acquisition and Appending this Data to the Dataframe
Next, latitudes and longitudes for the various neighborhoods are downloaded from https://public.opendatasoft.com/explore/dataset/us-zip-code-latitude-and-longitude/table/ as a .csv file.  These will be appended to the ZIP code dataframe.  

In [10]:
# The code was removed by Watson Studio for sharing.

In [11]:
#Read Latitude and Longitude .csv file.

Lat_Long = pd.read_csv(streaming_body_2, delimiter = ';')
Lat_Long.columns = ['ZIP Code', 'City', 'State', 'Latitude', 'Longitude', 'Time Zone', 'Daylight Savings Time Flag', 'Geopoint']
Lat_Long.head()

,ZIP Code,City,State,Latitude,Longitude,Time Zone,Daylight Savings Time Flag,Geopoint
0,71937,Cove,AR,34.398483,-94.39398,-6,1,"34.398483,-94.39398"
1,72044,Edgemont,AR,35.624351,-92.16056,-6,1,"35.624351,-92.16056"
2,56171,Sherburn,MN,43.660847,-94.74357,-6,1,"43.660847,-94.74357"
3,49430,Lamont,MI,43.010337,-85.89754,-5,1,"43.010337,-85.89754"
4,52585,Richland,IA,41.194129,-91.98027,-6,1,"41.194129,-91.98027"


In [12]:
#Remove City, State, Timezone, Daylight Savings Time Flag, and Geopoint from Lat_Long dataframe.

Lat_Long = Lat_Long.drop(columns=['City','State','Time Zone','Daylight Savings Time Flag', 'Geopoint'])
Lat_Long.head()

,ZIP Code,Latitude,Longitude
0,71937,34.398483,-94.39398
1,72044,35.624351,-92.16056
2,56171,43.660847,-94.74357
3,49430,43.010337,-85.89754
4,52585,41.194129,-91.98027


In [13]:
#Before merging the latitude and longitude data with the Charleston ZIP Code dataframe, check the datatypes of latitude and longitude data.  
#Since we are merging ZIP Code data, this ZIP Code data must be of type int64, like the ZIP Code data in our dataframe.

Lat_Long.dtypes

ZIP Code       int64
Latitude     float64
Longitude    float64
dtype: object

In [14]:
#Now we can merge the Charleston ZIP Code dataframe with the latitude and longitude data.
result = pd.merge(df,
                 Lat_Long[['ZIP Code', 'Latitude', 'Longitude']],
                 on = 'ZIP Code')
result

,ZIP Code,Latitude,Longitude
0,29401,32.779126,-79.93550
1,29402,32.848850,-79.85773
2,29403,32.799326,-79.94813
3,29404,32.897903,-80.06061
4,29405,32.856634,-79.98218
5,29406,32.918757,-80.02280
6,29407,32.794841,-80.00500
7,29409,32.848850,-79.85773
8,29412,32.737270,-79.95409
9,29413,32.848850,-79.85773


# Mapping the Neighborhoods in Charleston

In [15]:
#Import necessary libraries.

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # transform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.6.16          |           py36_1         149 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In order to define an instance of the geocoder, define a user_agent. We will name our agent <em>charleston_explorer</em>, as shown below.

In [16]:
address = 'Charleston, SC'

geolocator = Nominatim(user_agent="charleston_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates of Charleston are {}, {}.'.format(latitude, longitude))

The geographical coordinates of Charleston are 32.7876012, -79.9402728.


Create a map of Charleston with neighborhoods superimposed on top.

In [17]:
# create map of Charleston using latitude and longitude values
map_charleston = folium.Map(location=[latitude, longitude], zoom_start=11, control_scale = True)

# add markers to map
for lat, lng, z in zip(result['Latitude'], result['Longitude'], result['ZIP Code']):
    label = '{}'.format(z)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_charleston)  
    
map_charleston

## Exploring neighborhoods in Charleston.
### First, we will import Nearby Venues from Foursquare, based upon our location dataframe.

Define Foursquare Credentials and Version (Using Hidden Cell)

In [18]:
# The code was removed by Watson Studio for sharing.

Your credentials have been entered.


In [19]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Now write the code to run the above function on each neighborhood and create a new dataframe called *charleston_venues*.

In [20]:
charleston_venues = getNearbyVenues(names=result['ZIP Code'],
                                   latitudes=result['Latitude'],
                                   longitudes=result['Longitude']
                                    )

29401
29402
29403
29404
29405
29406
29407
29409
29412
29413
29414
29415
29416
29417
29418
29419
29422
29423
29424
29425
29426
29429
29439
29449
29451
29455
29457
29458
29464
29465
29466
29470
29482
29487


### How much data was retrieved for the Charleston area?  
#### We will check the total number of venues, and then we will return the total number of venues for each neighborhood.

In [21]:
#Checking size of resulting dataframe
print(charleston_venues.shape)
charleston_venues.head()

(307, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,29401,32.779126,-79.9355,Le Farfalle,32.780793,-79.934589,Italian Restaurant
1,29401,32.779126,-79.9355,Queen Street Grocery,32.777549,-79.936078,Food & Drink Shop
2,29401,32.779126,-79.9355,Second State Coffee,32.779859,-79.937648,Coffee Shop
3,29401,32.779126,-79.9355,Pure Barre Market Street,32.780714,-79.934621,Gym
4,29401,32.779126,-79.9355,Fulton Five,32.779654,-79.933429,Italian Restaurant


In [22]:
#Number of venues returned for each neighborhood.
charleston_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
29401,65,65,65,65,65,65
29402,4,4,4,4,4,4
29403,15,15,15,15,15,15
29404,4,4,4,4,4,4
29405,7,7,7,7,7,7
29406,3,3,3,3,3,3
29407,4,4,4,4,4,4
29409,4,4,4,4,4,4
29413,4,4,4,4,4,4


## Results: The largest number of venues are in ZIP Code 29424, with 81 total venues.  The second largest number of venues are in ZIP Code 29401, with 65 total venues.  Interestingly, these ZIP Codes are right next to each other on the Charleston peninsula.  

### Let's reduce the dataframe so that we are only looking at the venues in 29424 and 29401.  We will call this dataframe *tourism_venues*.

In [24]:
tourism_venues_1 = charleston_venues.loc[charleston_venues.Neighborhood == 29424]
tourism_venues_2 = charleston_venues.loc[charleston_venues.Neighborhood == 29401]
tourism_venues = tourism_venues_1.append(tourism_venues_2) 
tourism_venues.tail()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
60,29401,32.779126,-79.9355,Leyla Fine Lebanese Cuisine,32.783263,-79.934618,Middle Eastern Restaurant
61,29401,32.779126,-79.9355,Apple King Street,32.783382,-79.934854,Electronics Store
62,29401,32.779126,-79.9355,Chopsticks House,32.783331,-79.933750,Chinese Restaurant
63,29401,32.779126,-79.9355,M. Dumas & Sons,32.783126,-79.934436,Clothing Store
64,29401,32.779126,-79.9355,FIRE street food,32.783097,-79.934457,Asian Restaurant


### Create a dataframe that only has hotels in 29424 and 29401. 

In [25]:
hotels = tourism_venues[tourism_venues['Venue Category'] == 'Hotel'].reset_index()
hotels

,index,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,159,29424,32.783076,-79.93701,The Restoration on King,32.782271,-79.934160,Hotel
1,181,29424,32.783076,-79.93701,Belmond Charleston Place,32.780998,-79.932466,Hotel
2,195,29424,32.783076,-79.93701,Wentworth Mansion,32.779803,-79.939880,Hotel
3,196,29424,32.783076,-79.93701,King Charles Inn,32.782630,-79.932240,Hotel
4,11,29401,32.779126,-79.93550,Belmond Charleston Place,32.780998,-79.932466,Hotel
5,30,29401,32.779126,-79.93550,The Mills House Wyndham Grand Hotel,32.777803,-79.931394,Hotel
6,33,29401,32.779126,-79.93550,The Restoration on King,32.782271,-79.934160,Hotel
7,41,29401,32.779126,-79.93550,Wentworth Mansion,32.779803,-79.939880,Hotel
8,55,29401,32.779126,-79.93550,King Charles Inn,32.782630,-79.932240,Hotel


### Create a map of the hotels in 29424 and 29401.

In [29]:
# create map of Charleston using latitude and longitude values
map_29424 = folium.Map(location=[32.783076, -79.93701], zoom_start=18, control_scale = True)

#Create a list of latitude and longitude coordinate pairs.
locations = hotels[['Venue Latitude', 'Venue Longitude']]
locationlist = locations.values.tolist()
len(locationlist)

#add markers to map
for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], popup=hotels['Venue'][point]).add_to(map_charleston)
map_charleston

### Create a dataframe that has venues other than hotels in 29424 and 29401.

In [54]:
not_hotels = tourism_venues[tourism_venues['Venue Category'] != 'Hotel'].reset_index()
len(not_hotels)
not_hotels.head()

,index,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,137,29424,32.783076,-79.93701,Mellow Mushroom,32.783651,-79.934946,Pizza Place
1,138,29424,32.783076,-79.93701,Caviar & Bananas,32.783780,-79.936034,Deli / Bodega
2,139,29424,32.783076,-79.93701,CO,32.784552,-79.935394,Vietnamese Restaurant
3,140,29424,32.783076,-79.93701,Basic Kitchen,32.782181,-79.934875,Restaurant
4,141,29424,32.783076,-79.93701,Albert Simons Center for the Arts,32.784299,-79.936763,College Arts Building


#### It makes sense that there are 137 venues that are not hotels.  We started with 146 and took away 9 hotels.  

### Map venues along with hotels to visualize venue density near hotels.

In [31]:
# create map of Charleston using latitude and longitude values
map_29424 = folium.Map(location=[32.783076, -79.93701], zoom_start=18, control_scale = True)

#Create a list of latitude and longitude coordinate pairs for hotels and not_hotels.
locations = hotels[['Venue Latitude', 'Venue Longitude']]
locationlist = locations.values.tolist()
not_locations = not_hotels[['Venue Latitude', 'Venue Longitude']]
not_locationlist = not_locations.values.tolist()

# add markers to map - hotels are markers, other venues are circles
for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], popup=hotels['Venue'][point]).add_to(map_charleston)
for point in range(0, len(not_locationlist)):
    folium.CircleMarker(not_locationlist[point], radius=5, color = 'RED').add_to(map_charleston)  

map_charleston

## Examining the map tells us that the three best hotels based upon walking distance to attractions, are The Restoration on King, Belmond Charleston Place, and King Charles Inn.  

## Let's determine the centroid of the venues to find out which hotel is closest to the most venues.

In [57]:
#Determine the centroid of the venue latitudes by finding their mean.

not_hotels.mean()

index                       113.255474
Neighborhood              29413.927007
Neighborhood Latitude        32.781346
Neighborhood Longitude      -79.936349
Venue Latitude               32.782308
Venue Longitude             -79.934154
dtype: float64

### The centroid's coordinates are (32.781346, -79.936349).  We can map this to see which hotel is closest to the centroid.

In [58]:
# create map of Charleston using latitude and longitude values
map_29424 = folium.Map(location=[32.783076, -79.93701], zoom_start=18, control_scale = True)

# Add centroid to map
folium.Marker(
    location=[32.781346, -79.936349], popup='Ideal Hotel Location', icon=folium.Icon(color='green')).add_to(map_charleston)

map_charleston

##  The best hotel, based upon number of venues within walking distance, is The Restoration on King.
### Let's examine the venue categories nearby.

In [59]:
# one hot encoding
tourism_onehot = pd.get_dummies(tourism_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tourism_onehot['Neighborhood'] = tourism_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [tourism_onehot.columns[-1]] + list(tourism_onehot.columns[:-1])
tourism_onehot = tourism_onehot[fixed_columns]

tourism_onehot.head()

,Neighborhood,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bar,...,Shoe Store,Southern / Soul Food Restaurant,Sporting Goods Shop,Sports Bar,Thai Restaurant,Theater,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
137,29424,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
138,29424,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
139,29424,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
140,29424,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
141,29424,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [68]:
# Next, group rows by neighborhood and take the mean of the frequency of occurrence of each category
tourism_grouped = tourism_onehot.mean().reset_index()
tourism_grouped

,index,0
0,Neighborhood,29413.760274
1,American Restaurant,0.034247
2,Antique Shop,0.006849
3,Art Gallery,0.006849
4,Art Museum,0.006849
5,Arts & Crafts Store,0.006849
6,Asian Restaurant,0.013699
7,Athletics & Sports,0.006849
8,Bakery,0.006849
9,Bar,0.006849


In [65]:
#Confirming new size
tourism_grouped.shape

(62, 2)

## There are 62 different types of venues within walking distance of the three recommended hotels.  Enjoy your Charleston vacation!